# Mosa Happy Hour Analysis (Baseline vs Promo)

This notebook is the full, end-to-end version of the analysis. It includes:
- pipeline design and data cleaning
- baseline vs promo comparison design (quasi-experimental)
- metrics, lift, and bootstrap confidence intervals
- results tables, plots, and interpretation
- challenges, limitations, and next steps


## 1) Environment and paths


In [ ]:
from pathlib import Path
import sys

project_root = Path().resolve()
sys.path.append(str(project_root / 'src'))

project_root


## 2) Design overview

**Goal**: Determine whether Monday HH underperformance is mostly baseline traffic or promo appeal.

**Design (quasi-experimental)**:
- Use a pre-promo baseline period to estimate the normal Monday vs Wednesday gap.
- Compare baseline ratio (Mon/Wed) to the promo ratio (Mon/Wed) for the same HH window.
- Compute per-day lifts relative to baseline and compare lift differences.

**Key definitions**:
- HH window: 2:00-5:00pm local time
- Baseline period: 2025-08-29 to 2025-10-19
- Promo period: 2025-10-20 to 2025-12-31
- Primary focus: Monday vs Wednesday

**Interpretation**:
- If promo ratio is similar to baseline ratio and lifts are similar, the gap is mostly baseline traffic.
- If promo ratio is notably lower than baseline ratio or Monday lift is much lower, the Monday promo likely underperforms.


## 3) Configuration: Fresh Fruit Tea mapping

We use category and item-name rules from a config file so the mapping can be edited without code changes.


In [6]:
import json

config_path = project_root / 'config' / 'fresh_fruit_tea_mapping.json'
with config_path.open('r', encoding='utf-8') as f:
    mapping = json.load(f)

mapping


FileNotFoundError: [Errno 2] No such file or directory: '/Users/brandon/mosa-happy-hour-analysis/notebooks/config/fresh_fruit_tea_mapping.json'

## 4) Pipeline execution (raw -> processed)

This step:
- loads both baseline and promo raw exports
- standardizes column names
- parses timestamps (America/Los_Angeles)
- tags period and HH window
- flags Fresh Fruit Tea items
- outputs item- and order-level processed tables


In [ ]:
import happy_hour_pipeline

happy_hour_pipeline.main()


## 5) Analysis execution (daily aggregates + baseline adjusted summary)

This step:
- aggregates order-level data to day x HH window metrics
- computes baseline vs promo ratios
- computes lift per day and delta-lift
- bootstraps 95% CIs for Net Sales and Transactions/hour
- writes daily_aggregates.csv, summary_table.csv, answer_to_anna.txt


In [ ]:
import hh_analysis

hh_analysis.main()


## 6) Key outputs: summary tables


In [ ]:
import pandas as pd

summary_path = project_root / 'exports' / 'summary_table.csv'
answer_path = project_root / 'exports' / 'answer_to_anna.txt'

summary = pd.read_csv(summary_path)

print(answer_path.read_text())

sample = 'no_outliers' if (summary['sample'] == 'no_outliers').any() else 'all_days'
summary_focus = summary[(summary['sample'] == sample) & (summary['metric'].isin(['net_sales_hh', 'transactions_per_hour']))]
summary_focus


## 7) Daily aggregates preview


In [ ]:
daily_path = project_root / 'exports' / 'daily_aggregates.csv'
daily = pd.read_csv(daily_path)
daily.head(10)


## 8) Visualization outputs

These PNGs are saved to exports/plots and are used in the slide deck.


In [ ]:
import hh_visualizations

hh_visualizations.main()


In [ ]:
from IPython.display import Image, display

plots_dir = project_root / 'exports' / 'plots'
plot_files = [
    'ratio_net_sales_baseline_vs_promo.png',
    'ratio_tph_baseline_vs_promo.png',
    'hh_net_sales_by_day.png',
    'hh_total_items_by_day.png',
    'fruit_tea_quantity_by_day.png',
    'fruit_tea_net_sales_by_day.png',
    'fruit_tea_effective_price_by_day.png',
]

for name in plot_files:
    path = plots_dir / name
    if path.exists():
        display(Image(filename=str(path)))


## 9) Slide deck export

This builds the PPTX and includes the baseline vs promo slide.


In [ ]:
import hh_pptx

hh_pptx.main()


## 10) Interpretation guide for ratios and lifts

- **Mon/Wed ratio**: Monday metric divided by Wednesday metric for the HH window.
- **Baseline ratio**: ratio in the pre-promo period.
- **Promo ratio**: ratio in the promo period.
- **Lift**: (promo mean - baseline mean) / baseline mean, computed separately for Monday and Wednesday.

If the promo ratio is lower than the baseline ratio and Monday lift is lower than Wednesday lift, that points to weaker Monday promo appeal relative to baseline expectations.


## 11) Challenges and limitations

- **No randomization**: This is a quasi-experimental design. Results are directional, not causal proof.
- **Partial days**: Some days may have missing HH hours; those are flagged as partial.
- **Outliers**: Days with extreme values are flagged by IQR within each period/day.
- **Data coverage**: Baseline and promo periods are fixed; special events or closures can bias results.
- **Category mapping**: Fresh Fruit Tea identification depends on category and name matching rules.


## 12) Notes on reproducibility

- Bootstrap uses a fixed seed.
- All outputs are saved under exports/ and can be regenerated by rerunning this notebook.
